In [1]:
import torch
import numpy as np 
import pandas as pd
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.utils import data

In [22]:
df = pd.read_csv('https://raw.githubusercontent.com/PhantomBlade009/music_files/main/music_train.csv')
df.head()

,id,release_date,dating,violence,world/life,night/time,shake the audience,family/gospel,romantic,communication,obscene,music,movement/places,light/visual perceptions,family/spiritual,like/girls,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,age,genre,topic
0,0,2014.0,0.001049,0.000254,0.078057,0.282523,0.000000,0.000654,1.109899,3.484935,0.360515,0.016644,0.000000,0.001343,0.000102,0.000814,0.022435,0.001021,0.673433,0.753102,0.002560,0.001046,0.040567,48.364265,0.085714,4,2.0
1,1,1962.0,0.059132,NaN,NaN,0.001836,0.000737,0.000000,0.758840,0.000000,0.018112,0.658993,0.001137,0.000499,0.032456,0.002707,0.242728,0.001697,0.690740,0.678925,0.785480,0.001009,0.743203,28.200609,0.828571,1,7.0
2,2,1969.0,0.027451,0.085989,0.039760,0.001288,0.001424,0.000857,0.545408,0.016914,0.015750,0.277632,0.003961,0.257626,0.090351,0.000284,0.360503,0.000000,0.186294,0.404628,0.890527,0.060886,0.093003,13.145639,0.728571,4,NaN
3,3,2004.0,0.050993,NaN,0.000000,0.000477,0.008157,0.002800,0.000000,0.003703,2.603917,0.005438,0.000000,0.001282,0.057493,0.001102,0.307565,0.067061,0.499643,0.685085,0.640129,0.163850,0.648084,62.663152,0.228571,0,5.0
4,4,1992.0,0.000551,0.028702,0.000000,0.069748,0.008735,0.000625,0.014364,0.253022,0.007968,3.980988,0.001139,0.143952,0.001518,0.000174,0.047896,0.000000,0.407141,0.616982,0.030576,NaN,NaN,48.641741,0.400000,1,1.0


In [23]:
df.drop(columns= 'id', inplace = True)
df.head()

,release_date,dating,violence,world/life,night/time,shake the audience,family/gospel,romantic,communication,obscene,music,movement/places,light/visual perceptions,family/spiritual,like/girls,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,age,genre,topic
0,2014.0,0.001049,0.000254,0.078057,0.282523,0.000000,0.000654,1.109899,3.484935,0.360515,0.016644,0.000000,0.001343,0.000102,0.000814,0.022435,0.001021,0.673433,0.753102,0.002560,0.001046,0.040567,48.364265,0.085714,4,2.0
1,1962.0,0.059132,NaN,NaN,0.001836,0.000737,0.000000,0.758840,0.000000,0.018112,0.658993,0.001137,0.000499,0.032456,0.002707,0.242728,0.001697,0.690740,0.678925,0.785480,0.001009,0.743203,28.200609,0.828571,1,7.0
2,1969.0,0.027451,0.085989,0.039760,0.001288,0.001424,0.000857,0.545408,0.016914,0.015750,0.277632,0.003961,0.257626,0.090351,0.000284,0.360503,0.000000,0.186294,0.404628,0.890527,0.060886,0.093003,13.145639,0.728571,4,NaN
3,2004.0,0.050993,NaN,0.000000,0.000477,0.008157,0.002800,0.000000,0.003703,2.603917,0.005438,0.000000,0.001282,0.057493,0.001102,0.307565,0.067061,0.499643,0.685085,0.640129,0.163850,0.648084,62.663152,0.228571,0,5.0
4,1992.0,0.000551,0.028702,0.000000,0.069748,0.008735,0.000625,0.014364,0.253022,0.007968,3.980988,0.001139,0.143952,0.001518,0.000174,0.047896,0.000000,0.407141,0.616982,0.030576,NaN,NaN,48.641741,0.400000,1,1.0


In [27]:
df.iloc[:, 0:df.shape[1]-1] = df.iloc[:, 0:df.shape[1]-1].fillna(df.iloc[:,0:df.shape[1]-1].mean())
df['topic'] = df['topic'].fillna(df['topic'].mode()[0])
df.isnull().sum()

release_date                0
dating                      0
violence                    0
world/life                  0
night/time                  0
shake the audience          0
family/gospel               0
romantic                    0
communication               0
obscene                     0
music                       0
movement/places             0
light/visual perceptions    0
family/spiritual            0
like/girls                  0
sadness                     0
feelings                    0
danceability                0
loudness                    0
acousticness                0
instrumentalness            0
valence                     0
energy                      0
age                         0
genre                       0
topic                       0
dtype: int64

In [49]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
scaler = StandardScaler()
ohe = OneHotEncoder(sparse=False)

features_np_no_cat = df.drop(columns = ['genre', 'topic']).values
features_np_cat = df['topic'].values.reshape(-1,1)
labels_np = df['genre'].values.reshape(-1,1)

features_np_cat = ohe.fit_transform(features_np_cat)[:,1:8]
print(features_np_cat.shape)

features_np_std = scaler.fit_transform(features_np_no_cat)
features_np_std = np.column_stack((features_np_std, features_np_cat))
labels_np_std = (labels_np)

(17743, 7)


In [50]:
from sklearn.model_selection import train_test_split
features_std_train, features_std_test, labels_std_train, labels_std_test = train_test_split(features_np_std, labels_np_std)
print(features_std_train.shape, labels_std_train.shape, features_std_test.shape, labels_std_test.shape)

(13307, 30) (13307, 1) (4436, 30) (4436, 1)


In [51]:
features_train = torch.Tensor(features_std_train)
features_test = torch.Tensor(features_std_test)

# from sklearn.preprocessing import OneHotEncoder 
# ohe = OneHotEncoder(sparse = False) 
# encoded_labels = ohe.fit_transform(df['genre'].values.reshape(-1,1))

labels_train = torch.Tensor(labels_std_train)
labels_test = torch.Tensor(labels_std_test)

train_dataset = (features_train, labels_train)

print(features_train.shape, labels_train.shape, features_test.shape, labels_test.shape)

torch.Size([13307, 30]) torch.Size([13307, 1]) torch.Size([4436, 30]) torch.Size([4436, 1])


In [52]:
def batcher(dataset, batch_size):
  dataset = TensorDataset(*dataset)
  data_loader = DataLoader(dataset = dataset, batch_size = batch_size, shuffle = True)
  return data_loader

In [72]:
from torch import nn 
num_features = features_train.shape[1]
num_labels = 7
net = nn.Sequential( nn.Linear(num_features, 25),
                     nn.Sigmoid(),
                     nn.Linear(25, 25),
                     nn.Sigmoid(),
                     nn.Linear(25, num_labels),
                     nn.Sigmoid() ) 

In [54]:
loss = nn.CrossEntropyLoss( )

In [55]:
optimizer = torch.optim.Adam(net.parameters(), amsgrad=True )

In [56]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.])

In [60]:
batch_size = 256
batch = batcher(train_dataset, batch_size)
num_epochs = 500

for epoch in range(num_epochs):
  for X,y in batch:
    optimizer.zero_grad()
    l = loss(net(X), y.squeeze(1).long())
    l.backward()
    optimizer.step()

print(loss(net(features_train), labels_train.squeeze(1).long()))
print(loss(net(features_test), labels_test.squeeze(1).long()))

tensor(1.6223, grad_fn=<NllLossBackward>)
tensor(1.6920, grad_fn=<NllLossBackward>)


In [61]:
probs_train = net(features_train).detach().numpy()
predictions_train = np.zeros((probs_train.shape[0],1))

for i in range(probs_train.shape[0]):
  index = 0
  max_prob = probs_train[i,index] 
  for j in range(probs_train.shape[1]):
    if (probs_train[i,j]>max_prob):
      max_prob = probs_train[i,j] 
      index = j
  predictions_train[i] = index


probs_test = net(features_test).detach().numpy()
predictions_test = np.zeros((probs_test.shape[0],1))

for i in range(probs_test.shape[0]):
  index = 0
  max_prob = probs_test[i,index] 
  for j in range(probs_test.shape[1]):
    if (probs_test[i,j]>max_prob):
      max_prob = probs_test[i,j] 
      index = j
  predictions_test[i] = index



In [62]:
from sklearn.metrics import classification_report

print("Training Report...")
print(classification_report(predictions_train, labels_train))

print("Testing Report...")
print(classification_report(predictions_test, labels_test))

Training Report...
              precision    recall  f1-score   support

         0.0       0.13      0.34      0.19       801
         1.0       0.56      0.41      0.47      3512
         2.0       0.20      0.48      0.28       196
         3.0       0.39      0.41      0.40      1712
         4.0       0.48      0.36      0.41      4308
         5.0       0.46      0.45      0.45      1208
         6.0       0.32      0.40      0.36      1570

    accuracy                           0.39     13307
   macro avg       0.36      0.41      0.37     13307
weighted avg       0.44      0.39      0.41     13307

Testing Report...
              precision    recall  f1-score   support

         0.0       0.12      0.35      0.18       255
         1.0       0.51      0.39      0.44      1120
         2.0       0.27      0.51      0.36        84
         3.0       0.41      0.40      0.40       597
         4.0       0.47      0.35      0.40      1438
         5.0       0.42      0.44      0.

##Implementation With nn.Module style class


In [69]:
class NeuralNet(nn.Module):
    def __init__(self, num_features, hidden_sizes, num_labels):
        super(NeuralNet, self).__init__()
        
        self.num_features = num_features
        self.num_labels = num_labels
        self.hidden_size_1 = hidden_sizes[0]
        self.hidden_size_2 = hidden_sizes[1]

        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax()

        self.l_in = nn.Linear(num_features, self.hidden_size_1) 
        self.l1 = nn.Linear(self.hidden_size_1, self.hidden_size_2)  
        self.l2 = nn.Linear(self.hidden_size_2, num_labels)
        
    def forward(self, x):
        out = self.l_in(x)
        out = self.sigmoid(out)
        out = self.l1(out)
        out = self.sigmoid(out)
        out = self.l2(out)
        out = self.sigmoid(out)
        
        return out

In [77]:
net_class = NeuralNet(num_features, (25,25), num_labels)
optimizer_class = torch.optim.Adam(net_class.parameters(), amsgrad=True)
loss_class = nn.CrossEntropyLoss()

In [78]:
batch_size = 256
batch = batcher(train_dataset, batch_size)
num_epochs = 500

for epoch in range(num_epochs):
  for X,y in batch:
    optimizer_class.zero_grad()
    l = loss(net_class(X), y.squeeze(1).long())
    l.backward()
    optimizer_class.step()

print(loss(net_class(features_train), labels_train.squeeze(1).long()))
print(loss(net_class(features_test), labels_test.squeeze(1).long()))

tensor(1.6462, grad_fn=<NllLossBackward>)
tensor(1.6859, grad_fn=<NllLossBackward>)
